In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
from datasets import Dataset
from datasets import load_dataset

load_dotenv(override=True)

/home/jeal/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [5]:
# Load from hub
ds_questions = load_dataset(
    "jealk/dk_retrieval_benchmark",
    "generated_questions",
    split="train",
    download_mode="force_redownload",
)

Generating train split: 100%|██████████| 200/200 [00:00<00:00, 6221.99 examples/s]


In [6]:
ds_questions

Dataset({
    features: ['title_vejledning', 'chunk_text', 'url', 'generated_question'],
    num_rows: 200
})

In [13]:
df_questions = ds_questions.to_pandas()
df_questions.head()

,title_vejledning,chunk_text,url,generated_question
0,Vejledning om regulering af satser fra 1. janu...,Vejledning om regulering af satser fra 1. janu...,https://www.retsinformation.dk/eli/retsinfo/20...,Hvem fastsætter reguleringen af satserne for a...
1,Vejledning om regulering af satser fra 1. janu...,Vejledning om regulering af satser fra 1. janu...,https://www.retsinformation.dk/eli/retsinfo/20...,Med hvilken procent vil satserne for arbejdssk...
2,Vejledning om regulering af satser fra 1. janu...,Tilpasningsprocenten er ved Finansministeriets...,https://www.retsinformation.dk/eli/retsinfo/20...,Hvad er tilpasningsprocenten for finansåret 20...
3,Vejledning om regulering af satser fra 1. janu...,Tilpasningsprocenten er ved Finansministeriets...,https://www.retsinformation.dk/eli/retsinfo/20...,Fra hvilken dato træder de nye satser i lov om...
4,Vejledning om regulering af satser fra 1. janu...,1541 af 12. december 2023 fastsættes løbende e...,https://www.retsinformation.dk/eli/retsinfo/20...,"Fra hvilken dato vil arbejdsskader, der sker, ..."


In [14]:
# Rename and drop columns to fit promptlayer evaluation
# Drop generated_question
df_questions = df_questions.drop(columns=["generated_question"])
# Add a column num_questions with the value 1
df_questions["num_questions"] = 1
# Delete every second row
df_questions = df_questions[::2]

In [15]:
# Export the dataframe as a csv (maintaining UTF8 encoding)
df_questions.to_csv("data/data/generated_questions.csv", index=False, encoding="utf-8")

# Egnet chunk?

In [25]:
from llama_index.prompts import PromptTemplate

# Define your custom prompt template in Danish
qa_egnet_tmlp = """ Din opgave er at vurdere hvorvidt et uddrag af en tekst indeholder faktuel information som gør uddraget egnet til at stille et generelt spørgsmål til. 
============================
Start på eksempel som skal have scoren 1:

"Vejledning om regulering af satser fra 1. januar 2024 efter lov om arbejdsskadesikring, lov om sikring mod følger af arbejdsskade, lov om arbejdsskadeforsikring og lov om forsikring mod følger af ulykkestilfælde Indledning Efter lov om arbejdsskadesikring, jf. lovbekendtgørelse nr. 1186 af 19. august 2022 med de ændringer, der følger af lov nr. 1541 af 12. december 2023, og lov om sikring mod følger af arbejdsskade, jf. lovbekendtgørelse nr. 943 af 16. oktober 2000, skal der med virkning fra 1. januar 2024 efter indstilling fra bestyrelsen for Arbejdsmarkedets Erhvervssikring ske regulering af lovens årslønsbeløb, godtgørelsesbeløb, overgangsbeløb samt løbende erstatninger. Reguleringen af satserne fastsættes af Arbejdstilsynets direktør efter bemyndigelse fra beskæftigelsesministeren. Satser efter loven reguleres med 2 procent tillagt tilpasningsprocenten for finansåret 2024 (jf. lov om en satsreguleringsprocent)."

Fordi uddraget indeholder klare og faktuelle informationer, hvorfra der kan formuleres et præcist, naturligt og kort spørgsmål der kan besvares ud fra uddraget, eksempelvis

"Hvem fastsætter reguleringen af satserne for arbejdsskadesikring og andre relaterede ydelser?"

============================
Start på eksempel som skal have scoren 0:

"årligt. 8)Uddannelsesgodtgørelsen i en forlænget periode efter § 18 b, stk. 3, 2. pkt., udgør 244.140 kr. årligt. Grundlønnen for beregning og regulering af løbende erstatning og uddannelsesgodtgørelse for arbejdsskader indtruffet den 1. juli 2024 eller senere er den efter lov om arbejdsskadesikring § 24 fastsatte årsløn multipliceret med 608.000/608.000, jf. § 24 a. Fastsættes en løbende erstatning eller en uddannelsesgodtgørelse den 1. juli 2024 eller senere, udbetales erstatningen eller godtgørelsen fra tidspunktet for dennes begyndelse med et tillæg på 0,0 pct. til den erstatning eller godtgørelse, der svarer til grundlønnen. Satser for arbejdsskader indtruffet i tiden 1. januar 2024 til 30. juni 2024 Med virkning for arbejdsskader efter lov om arbejdsskadesikring, jf. lovbekendtgørelse nr. 1186 af 19. august 2022 med de ændringer, der følger af lov nr."

Fordi der uddraget ikke indeholder meget detaljerede informationer der er svære at forstå uden kontekst, generelt er mere fragmenteret og gør det vanskeligt at formulere et generelt spørgsmål.

==============================
Din opgave, hvordan du skal evaluere et uddrag:

Såfremt der kan opstilles et naturligt formuleret og faktuelt spørgsmål til uddraget nedenfor, som man kunne forestille sig at bruge i sammenhæng med en eksamen eller test, skal du give scoren 1 til teksten. 

Hvis uddraget ikke indeholder generel faktuel information hvorfra man kunne formulere et naturligt spørgsmål, giv da uddraget en score på 0.

==============================
Uddrag som du skal score, 0 eller 1:

{chunk_text}"

Returner KUN tallet 0 eller 1, ingen yderligere forklaring

"""

qa_egnet_tmlp = PromptTemplate(qa_egnet_tmlp)

In [34]:
# Return chunk_text from df_questions as a list
chunk_text = df_questions["chunk_text"].tolist()

chunk_text[15]

'Arbejdstilsynet, den 5. januar 2024\nSine Frederiksen\n/ Helle Klostergaard Christensen\nBilag 1\nBilaget indeholder eksempler på beregninger af kapitalerstatninger, godtgørelsesbeløb og overgangsbeløb samt løbende erstatninger og godtgørelser, som tilskadekomne eller dennes efterladte har ret til efter lov om arbejdsskadesikring, lov om sikring mod følger af arbejdsskade, lov om arbejdsskadeforsikring og lov om forsikring mod følger af ulykkestilfælde.\nArbejdsskader indtruffet i tiden 1. juli 2024 til 31. december 2024\nTil brug ved beregning og fremtidig regulering af løbende erstatninger for tab af erhvervsevne og tab af forsørger samt uddannelsesgodtgørelser beregnes grundlønnen (§ 24 a) ved at multiplicere den fastsatte årsløn med forholdet mellem den maksimale årsløn pr. 1. januar 2024 (608.000 kr.) og den maksimale årsløn på skadestidspunktet (608.000 kr.), det vil sige:\nGrundløn = årsløn × 608.000 / 608.000.'

In [35]:
full_prompt = qa_egnet_tmlp.format(chunk_text=chunk_text[15])

In [36]:
chunk_text_to_evaluate = "Her er et eksempel på tekst, der skal evalueres."
from openai import OpenAI

client = OpenAI()


def evaluate_chunk(chunk_text):
    full_prompt = qa_egnet_tmlp.format(chunk_text=chunk_text)
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "Du er en erfaren sagsbehandler, der skal evaluere et uddrag af en tekst. Uddraget skal vurderes på hvorvidt det det er egnet til at stille et generelt spørgsmål til. ",
            },
            {"role": "user", "content": full_prompt},
            # If you need to provide additional context or instructions, you can add more messages here
        ],
    )
    return completion.choices[0].message.content

In [37]:
eval_list_score = []
for chunk in chunk_text:
    eval_list_score.append(evaluate_chunk(chunk))

In [40]:
from collections import Counter

Counter(eval_list_score)

Counter({'1': 51, '0': 49})

In [39]:
# Append the scores to the dataframe
df_questions["score"] = eval_list_score
df_questions.head()

,title_vejledning,chunk_text,url,num_questions,score
0,Vejledning om regulering af satser fra 1. janu...,Vejledning om regulering af satser fra 1. janu...,https://www.retsinformation.dk/eli/retsinfo/20...,1,1
2,Vejledning om regulering af satser fra 1. janu...,Tilpasningsprocenten er ved Finansministeriets...,https://www.retsinformation.dk/eli/retsinfo/20...,1,1
4,Vejledning om regulering af satser fra 1. janu...,1541 af 12. december 2023 fastsættes løbende e...,https://www.retsinformation.dk/eli/retsinfo/20...,1,0
6,Vejledning om regulering af satser fra 1. janu...,"og det maksimale årslønsbeløb, der var gældend...",https://www.retsinformation.dk/eli/retsinfo/20...,1,0
8,Vejledning om regulering af satser fra 1. janu...,Satser for arbejdsskader indtruffet 1. juli 20...,https://www.retsinformation.dk/eli/retsinfo/20...,1,1


In [45]:
# Save 5 examples of chunk_text with score 1 in a list of strings
examples1 = df_questions[df_questions["score"] == "1"].head(5)["chunk_text"].tolist()
# and of 0s
examples0 = df_questions[df_questions["score"] == "0"].head(5)["chunk_text"].tolist()

In [49]:
print(examples1[0])

Vejledning om regulering af satser fra 1. januar 2024 efter lov om arbejdsskadesikring, lov om sikring mod følger af arbejdsskade, lov om arbejdsskadeforsikring og lov om forsikring mod følger af ulykkestilfælde
Indledning
Efter lov om arbejdsskadesikring, jf. lovbekendtgørelse nr. 1186 af 19. august 2022 med de ændringer, der følger af lov nr. 1541 af 12. december 2023, og lov om sikring mod følger af arbejdsskade, jf. lovbekendtgørelse nr. 943 af 16. oktober 2000, skal der med virkning fra 1. januar 2024 efter indstilling fra bestyrelsen for Arbejdsmarkedets Erhvervssikring ske regulering af lovens årslønsbeløb, godtgørelsesbeløb, overgangsbeløb samt løbende erstatninger.
Reguleringen af satserne fastsættes af Arbejdstilsynets direktør efter bemyndigelse fra beskæftigelsesministeren.
Satser efter loven reguleres med 2 procent tillagt tilpasningsprocenten for finansåret 2024 (jf. lov om en satsreguleringsprocent).


In [50]:
# Create a CSV export of the dataframe for all rows that have a score of 1
df_questions[df_questions["score"] == "1"].to_csv(
    "data/data/generated_questions_score1.csv", index=False, encoding="utf-8"
)

# Create a CSV export of the dataframe for all rows that have a score of 0
df_questions[df_questions["score"] == "0"].to_csv(
    "data/data/generated_questions_score0.csv", index=False, encoding="utf-8"
)

## Genererede spørgsmål
Eksempler fra promptflow

**Ud fra chunks Scoret 1**

Hvordan bliver satserne for arbejdsskadesikring justeret fra 1. januar 2024?

Hvad er den nye tilpasningsprocent for finansåret 2024 ifølge Finansministeriets bekendtgørelse?

Hvad er det maksimale beløb, man kan modtage for årsløn ifølge de nye satser for arbejdsskader indtruffet fra 1. juli 2024?

Hvad er det fastsatte maksimumbeløb for årslønnen ifølge den nye lovændring?



**Ud fra chunks Scoret 0**

Hvordan påvirker den nye lov nr. 1541 af 12. december 2023 erstatninger og uddannelsesgodtgørelser for arbejdsskader, der sker før og efter 1. juli 2024?

Hvordan ændres erstatnings- og uddannelsesgodtgørelsesniveauet for arbejdsskader indtruffet før 1. juli 2024 fra 1. januar 2024?

Hvad er den årlige uddannelsesgodtgørelse for en forlænget periode efter § 18 b, stk. 3, 2. pkt.?

Hvad er den årlige uddannelsesgodtgørelse i en forlænget periode efter § 18 b, stk. 3, 2. pkt.?
    
    